# RAISE the Bar: Restriction of Action Spaces for Improved Social Welfare and Equity in Traffic Management 
## Generalized Braess graph experiment

### Setup and function definitions

In [ ]:
# Internal modules
import math

# External modules
import pandas as pd
import networkx as nx
import matplotlib as mpl
import matplotlib.pyplot as plt
import matplotlib.ticker as mtick
from tqdm import tqdm

# Own modules
import os, sys
sys.path.append(f'{os.getcwd()}/../../')

from src.environment import TrafficModel, create_cars, build_network
from src.analysis import compute_regression, analyze_fairness
from src.util import change_value_of_money

plt.rcParams['text.usetex'] = True

In [ ]:
def create_multiple_braess_network(k=0, capacity=100):
    network = nx.DiGraph(
        [('S0', 'A'), ('S0', 'B'), ('A', 'B'), ('A', 'T0'), ('B', 'T0')]
        + [(f'S{i}', f'S{i-1}') for i in range(1, k + 1)]
        + [(f'S{i}', 'B') for i in range(1, k + 1)]
        + [(f'S{i-1}', f'T{i}') for i in range(1, k + 1)]
        + [('B', f'T{i}') for i in range(1, k + 1)]
    )

    nx.set_node_attributes(
        network,
        {
            **{'A': (3 * math.cos(-1 * 2 * math.pi / (2*k+4)), 3 * math.sin(-1 * 2 * math.pi / (2*k+4))), 'B': (0, 0)},
            **{f'S{i}': (3 * math.cos((2*i+1) * 2 * math.pi / (2*k+4)), 3 * math.sin((2*i+1) * 2 * math.pi / (2*k+4))) for i in range(0, k + 1)},
            **{f'T{i}': (1.5 * math.cos(2*i * 2 * math.pi / (2*k+4)), 1.5 * math.sin(2*i * 2 * math.pi / (2*k+4))) for i in range(0, k + 1)}
        },
        "position",
    )

    nx.set_edge_attributes(
        network,
        {
            ('S0', 'A'): (2, 6, capacity, 1), ('S0', 'B'): (10, 0, 1, 1), ('A', 'B'): (1, 0, 1, 1), ('A', 'T0'): (10, 0, 1, 1), ('B', 'T0'): (2, 6, capacity, 1),
            **{(f'S{i}', f'S{i-1}'): (2, 6, capacity, 1) for i in range(1, k + 1)},
            **{(f'S{i}', 'B'): (11 + 2*i, 0, 1, 1) for i in range(1, k + 1)},
            **{(f'S{i-1}', f'T{i}'): (11 + 2*i, 0, 1, 1) for i in range(1, k + 1)},
            **{('B', f'T{i}'): (2, 6, capacity, 1) for i in range(1, k + 1)}
        },
        "latency_params",
    )

    return build_network(network)

In [ ]:
BASE_COLOR = 'aqua'
RESTRICTION_COLOR = 'coral'
TOLLING_COLOR = 'darkred'

### Experiment

In [ ]:
k = 20
capacity = 20
number_of_steps = 1_000
values_of_money = [0, 1, 2, 5]

demands = range(k + 1)
results = []
for demand in tqdm(demands):
    car_counts = {(f'S{demand}', f'T{demand}'): 50}
    
    # Unrestricted
    network = create_multiple_braess_network(k=k, capacity=capacity)
    cars = create_cars(network, car_counts=car_counts)
    change_value_of_money(cars, values_of_money)
    model = TrafficModel(network, cars)
    step_stats_unrestricted, car_stats_unrestricted = model.run_sequentially(number_of_steps, show_progress=False)
    results.append({'demand': demand, 'scenario': 'Base', 'travel_time': car_stats_unrestricted["travel_time"].mean(), **analyze_fairness(car_stats_unrestricted)})

    # Restricted
    network = create_multiple_braess_network(k=k, capacity=capacity)
    cars = create_cars(network, car_counts=car_counts)
    change_value_of_money(cars, values_of_money)
    model = TrafficModel(network, cars)

    model.set_edge_restriction(('A', 'B'), False)
    for i in range(1, min(demand, k) + 1):
        model.set_edge_restriction((f'S{i-1}', 'B'), False)

    step_stats_restricted, car_stats_restricted = model.run_sequentially(number_of_steps, show_progress=False)
    results.append({'demand': demand, 'scenario': 'Restriction', 'travel_time': car_stats_restricted["travel_time"].mean(), **analyze_fairness(car_stats_restricted)})

    # Tolling
    network = create_multiple_braess_network(k=k, capacity=capacity)
    cars = create_cars(network, car_counts=car_counts)
    change_value_of_money(cars, values_of_money)
    model = TrafficModel(network, cars, tolls=True, beta=1, R=0.1)
    step_stats_tolling, car_stats_tolling = model.run_sequentially(number_of_steps, show_progress=False)
    results.append({'demand': demand, 'scenario': 'Tolling (excl. tolls)', 'travel_time': car_stats_tolling["travel_time"].mean(), **analyze_fairness(car_stats_tolling)})
    results.append({'demand': demand, 'scenario': 'Tolling (incl. tolls)', 'travel_time': car_stats_tolling["total_cost"].mean()})

results = pd.DataFrame(results).set_index(['demand', 'scenario']).unstack(level=1)

In [ ]:
performance = results['travel_time']
ax = performance.plot(color=[BASE_COLOR, RESTRICTION_COLOR, TOLLING_COLOR, TOLLING_COLOR], style=['-', '-', '-', '--'])
ax.legend()
ax.set_xlabel('Demand (S_i -> T_i)')
ax.set_ylabel('Mean total cost')
ax.xaxis.get_major_locator().set_params(integer=True)

plt.tight_layout()
ax.get_figure().savefig('braess-absolute-performance.pdf', dpi=300)

In [ ]:
relative_improvement = -(performance.drop('Base', axis=1).sub(performance['Base'], axis=0).div(performance['Base'], axis=0))
ax = relative_improvement.plot(color=[RESTRICTION_COLOR, TOLLING_COLOR, TOLLING_COLOR], style=['-', '-', '--'])
ax.legend()
ax.set_xlabel(r'Demand $(s_i \rightarrow t_i)$')
ax.set_ylabel(r'Improvement relative to \emph{Base}')
ax.axhline(ls='--', color='grey')
ax.xaxis.get_major_locator().set_params(integer=True)
ax.yaxis.set_major_formatter(mtick.PercentFormatter(1.0))

plt.tight_layout()
ax.get_figure().savefig('braess-relative-performance.pdf', dpi=300)

In [ ]:
fairness = results['slope'].drop('Tolling (incl. tolls)', axis=1)
fairness.columns = ['Base', 'Restriction', 'Tolling']

ax = fairness.plot(color=[BASE_COLOR, RESTRICTION_COLOR, TOLLING_COLOR])
ax.legend()
ax.set_xlabel(r'Demand $(s_i \rightarrow t_i)$')
ax.set_ylabel('Slope of regression')
ax.axhline(ls='--', color='grey')
ax.xaxis.get_major_locator().set_params(integer=True)

plt.tight_layout()
ax.get_figure().savefig('braess-fairness.pdf', dpi=300)

### Detailed fairness plot for $B_1$

In [ ]:
k = 1
capacity = 20
number_of_steps = 10_000
values_of_money = [0, 1, 2, 5]
car_counts = {(f'S{k}', f'T{k}'): 50}
    
# Unrestricted
network = create_multiple_braess_network(k=k, capacity=capacity)
cars = create_cars(network, car_counts=car_counts)
change_value_of_money(cars, values_of_money)
model = TrafficModel(network, cars)
step_stats_unrestricted, car_stats_unrestricted = model.run_sequentially(number_of_steps, show_progress=False)

# Restricted
network = create_multiple_braess_network(k=k, capacity=capacity)
cars = create_cars(network, car_counts=car_counts)
change_value_of_money(cars, values_of_money)
model = TrafficModel(network, cars)

model.set_edge_restriction(('A', 'B'), False)
for i in range(1, k + 1):
    model.set_edge_restriction((f'S{i-1}', 'B'), False)

step_stats_restricted, car_stats_restricted = model.run_sequentially(number_of_steps, show_progress=False)

# Tolling
network = create_multiple_braess_network(k=k, capacity=capacity)
cars = create_cars(network, car_counts=car_counts)
change_value_of_money(cars, values_of_money)
model = TrafficModel(network, cars, tolls=True, beta=1, R=0.1)
step_stats_tolling, car_stats_tolling = model.run_sequentially(number_of_steps, show_progress=False)

In [ ]:
cutoff = 5_000

u = car_stats_unrestricted[car_stats_unrestricted['step'] >= cutoff].groupby('value_of_money')['travel_time'].mean()
r = car_stats_restricted[car_stats_restricted['step'] >= cutoff].groupby('value_of_money')['travel_time'].mean()
t = car_stats_tolling[car_stats_tolling['step'] >= cutoff].groupby('value_of_money')['travel_time'].mean()

travel_times = pd.concat([u, r, t], keys=['Base', 'Restriction', 'Tolling'], axis=1)

ax = travel_times.transpose().plot(kind='bar', cmap=mpl.colormaps['winter'])
ax.legend(loc='lower left', title='Value of money')
plt.xticks(rotation = 0)
ax.set_ylabel('Travel time')

# Add regression lines
for offset, data in enumerate([u, r, t]):
    slope, intercept, error, p = compute_regression(data.reset_index())
    ax.plot([offset - 0.3, offset + 0.3], [intercept + slope * min(values_of_money), intercept + slope * max(values_of_money)], 'r--')

plt.tight_layout()
ax.get_figure().savefig('braess-detailed-fairness.pdf', dpi=300)